In [1]:

# ========================== 📦 IMPORT SECTION ==========================
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import time
from ast import literal_eval
from typing import List, Dict

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

from ragas.dataset_schema import Sample, EvaluationDataset, SingleTurnSample
from src.tag.src.text2sqlchain import generate_sql  # path kamu
from src.tag.evaluation.run_text2sql import run_text2sql_workflow
from src.tag.src.text2sqlchain import generate_sql
from src.tag.src.query_executor import execute_text2sql_response
from src.tag.evaluation.eval_metrics import evaluate_retriever  # gunakan retriever-style
from src.tag.database.db_connection import connect_db
from src.tag.database.schema_loader import load_schema

from langchain_community.utilities import SQLDatabase



d:\code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conn = connect_db()
schema = load_schema(conn)

In [3]:
DATASET_PATH = os.path.join("data", "Dataset Testing 2.xlsx")
df = pd.read_excel(DATASET_PATH)

dataset = []

for i, row in df.iterrows():
    if row["is_valid"]:
        dataset.append({
            "user_input": str(row["user_input"]),
            "reference_contexts": literal_eval(row["reference_contexts_2"])
        })
    if len(dataset) == 5:
        break
print(f"Jumlah soal: {len(dataset)}")


Jumlah soal: 5


In [5]:
def retrieve_contexts_from_text2sql(question: str) -> list[str]:
    try:
        response = generate_sql(schema, question, top_k=100, shot_mode="few-shot", llm_mode="gemini")
        rows, columns = execute_text2sql_response(conn, response)
        if not rows:
            return []
        return [" | ".join(map(str, row)) for row in rows]
    except Exception as e:
        print(f"[!] Error: {question} → {e}")
        return []


In [6]:
samples = []

for item in tqdm(dataset, desc="Menjalankan Text2SQL dan Eksekusi"):
    q = item["user_input"]
    ref = item["reference_contexts"]
    ret = retrieve_contexts_from_text2sql(q)

    sample = SingleTurnSample(
        question=q,
        reference_contexts=ref,
        retrieved_contexts=ret
    )
    samples.append(sample)

evaluation_dataset = EvaluationDataset(samples)


Menjalankan Text2SQL dan Eksekusi:   0%|          | 0/5 [00:00<?, ?it/s]d:\code\src\tag\src\text2sqlchain.py:126: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)
d:\code\src\tag\src\text2sqlchain.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run(inputs).strip()
Menjalankan Text2SQL dan Eksekusi: 100%|██████████| 5/5 [00:08<00:00,  1.61s/it]


In [7]:
result = evaluate_retriever(evaluation_dataset, experiment_name="tag_retriever_v1")
df_result = result.to_pandas()
df_result


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]Exception raised in Job[5]: ValueError(max() arg is an empty sequence)
Exception raised in Job[3]: ValueError(max() arg is an empty sequence)
Evaluating: 100%|██████████| 10/10 [00:00<00:00, 60.78it/s]


,retrieved_contexts,reference_contexts,precision,recall
0,[18A | (1) Dalam penyediaan akses dan layanan ...,[(1) Persentase TKDN untuk belanja modal (cape...,0.230769,0.75
1,[],[Lembaga Penyiaran Asing dilarang didirikan di...,0.000000,NaN
2,[],[(1) Pelaksanaan Diklat REOR sebagaimana dimak...,0.000000,NaN
3,[Jaringan Dokumentasi dan Informasi Hukum Keme...,[Dalam Peraturan Menteri ini yang dimaksud den...,0.000000,0.00
4,[22 | (1) Registrar Nama Domain Selain Instans...,[Registri Nama Domain dan Registrar Nama Domai...,0.000000,0.00


In [9]:
df_result.to_json("tag_retriever_v1.json", index=False)
print("Hasil evaluasi telah disimpan ke 'tag_retriever_v1.json'.")

Hasil evaluasi telah disimpan ke 'tag_retriever_v1.json'.
